# Calculate Cross Dataset Performance

In [ ]:
from dotenv import load_dotenv
import os
import pandas as pd
load_dotenv()
import pickle as pk
import numpy as np
from pathlib import Path
base_dir = os.getenv('BASEDIR')

from tqdm import tqdm
tqdm.pandas()

# from lightgbm import LGBMClassifier
from flaml.default import LGBMClassifier
from itertools import chain, combinations

import pandas as pd
import os 
import re
import numpy as np
from tqdm import tqdm

from sklearn.model_selection import cross_val_predict, StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.datasets import make_classification

import os
from collections import defaultdict

In [ ]:
datasets = ['qanda','ausvotes','socialsense','riot','parler']

In [ ]:
performance = {dataset : {} for dataset in datasets}
gt_f1 = {dataset : {} for dataset in datasets}

In [ ]:
import itertools
def all_combinations(l):
    return(itertools.product(l, repeat=2))

In [ ]:
granularity = '_per_user'
feature_set = ['use']
ground_truth = 'URL_LR'
for train, test in itertools.chain(all_combinations(datasets)):
    train_path = os.path.join(base_dir,'data','03_processed',train,'ground_truth', train+'_'+ground_truth+'_'+granularity+'.pk')
    if not os.path.exists(train_path):
        continue
    test_path = os.path.join(base_dir,'data','03_processed',test,'ground_truth', test+'_'+ground_truth+'_'+granularity+'.pk')
    if not os.path.exists(test_path):
        continue
#         print(train, test)
    def load_vector(fp):
        with open(fp, 'rb') as rf:
            vec = pk.load(rf)
        return vec
    
    y_train = load_vector(train_path).values
    mask_train = y_train >= 0

    y_test = load_vector(test_path).values
    mask_test = y_test >= 0
    if any(mask_train):
        y_train = y_train[mask_train]
        feature_paths = [os.path.join(base_dir,'data','03_processed',train,'features', train+'_'+feature+'_'+granularity+'.pk') for feature in feature_set]
        X_train = np.asarray(np.hstack([load_vector(fp) for fp in feature_paths]))
        X_train = X_train[mask_train]

        y_test = y_test[mask_test]
        feature_paths = [os.path.join(base_dir,'data','03_processed',test,'features', test+'_'+feature+'_'+granularity+'.pk') for feature in feature_set]
        X_test = np.asarray(np.hstack([load_vector(fp) for fp in feature_paths]))
        X_test = X_test[mask_test]

        et = LGBMClassifier(is_unbalance=True,n_estimators=200, n_jobs=-1, verbose=-1, seed=123)
        if train == test:
            skf = StratifiedKFold(n_splits=5, shuffle=True)
            preds = cross_val_predict(et, X_train, y_train, cv=skf, method='predict_proba')
        else:
            et.fit(X_train,y_train)
            preds = et.predict_proba(X_test)
        # evaluating the model
        print('Area under the ROC Curve:', roc_auc_score(y_test, preds, multi_class='ovo'))

        performance[train][test] = roc_auc_score(y_test, preds, multi_class='ovo')

In [ ]:
col_order = datasets
performance_df =  pd.DataFrame(performance).reindex(index=col_order, columns=col_order) 

In [ ]:
performance_df.to_csv(os.path.join(base_dir, 'data', '04_results', 'cross_dataset_auc.csv'))